In [2]:
from py2neo import Graph, Node, Relationship
import pandas as pd
from IPython.display import Javascript
# from fuzzywuzzy import process, fuzz

In [3]:
import platform
import psutil
import os

In [4]:
conda_version = ! conda -V
print(f"conda : {conda_version[0].split()[1]}")
pip_version = ! pip -V
print(f"pip   : {pip_version[0].split()[1]}")
python_version = ! python -V
print(f"python: {python_version[0].split()[1]}")
pandas_version = ! pip list |grep -i pandas
print(f"pandas: {pandas_version[0].split()[1]}")
py2neo_version = ! pip list |grep -i py2neo
print(f"py2neo: {py2neo_version[0].split()[1]}")

conda : 4.9.2
pip   : 20.2.4
python: 3.8.6
pandas: 1.1.4
py2neo: 4.2.0


In [34]:
server  = "13.93.37.217"
port    = "7687"
user    = "neo4j"
passw   = "digityser"
db_name = "neo4j"        # default name (v4.x)

In [35]:
# load graph connection instance
graph = Graph(host=server, auth=(user, passw), name=db_name, encrypted=False)

In [33]:
# import the dataset
synthetic_df = pd.read_csv("../../data/bronze/synthetic_data.csv")

### data cleaning prep

In [16]:
synthetic_df["id"] = synthetic_df.index

### data loading Graph DB

In [17]:
# save file to /import
file = "synthetic_data.csv"
synthetic_df.to_csv("../../neo4j/import/"+file, sep=',', index=False)

In [7]:
graph.run("CREATE CONSTRAINT ON (p:Patient) ASSERT p.id IS UNIQUE")

In [8]:
graph.run("CREATE CONSTRAINT ON (r:Race) ASSERT r.value IS UNIQUE")

In [29]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (p:Patient {id:(line.id)})
MERGE (r:Race {value:(line.race)})
MERGE (p)-[:IS_RACE]->(r)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 78446
labels_removed: 0
nodes_created: 78446
nodes_deleted: 0
properties_set: 78446
relationships_created: 78441
relationships_deleted: 0

In [36]:
graph.run("CREATE CONSTRAINT ON (g:Gender) ASSERT g.value IS UNIQUE")

In [39]:
#status queries
graph.run("""CALL dbms.listQueries()""").to_data_frame()[["queryId", "query", "status", "elapsedTimeMillis"]].T

,0,1,2
queryId,query-3270,query-3279,query-3238
query,\nUSING PERIODIC COMMIT 1000\nLOAD CSV WITH HE...,CALL dbms.listQueries(),\nUSING PERIODIC COMMIT 1000\nLOAD CSV WITH HE...
status,waiting,running,running
elapsedTimeMillis,52988,54,166961


In [42]:
#remove queries
graph.run("""CALL dbms.killQueries(["query-3238"])""").to_data_frame()

,queryId,username,message
0,query-3238,neo4j,Query found


In [43]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (p:Patient {id:(line.id)})
MERGE (g:Gender {value:(line.gender)})
MERGE (p)-[:HAS_GENDER]->(g)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 2
labels_removed: 0
nodes_created: 2
nodes_deleted: 0
properties_set: 2
relationships_created: 78441
relationships_deleted: 0

In [45]:
graph.run("CREATE CONSTRAINT ON (ag:AgeGroup) ASSERT ag.value IS UNIQUE")

In [47]:
file

'synthetic_data.csv'

In [48]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (p:Patient {id:(line.id)})
MERGE (ag:AgeGroup {value:(line.age)})
MERGE (p)-[:HAS_AGE]->(ag)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 78441
relationships_deleted: 0